In [28]:
f = open('./preprocess/bilinear.txt','r')
text_emb = []
for line in f:
    line_text = [ord(i) for i in line]
    while len(line_text) < 80: #padding
        line_text.insert(0,0)
    text_emb.append(line_text)
text_emb[1]
len(text_emb[2])
    
# for lines in f:
#     [ord(i) for i in 'def lmul : A →ₐ[R] (End R A) :=']

80

In [46]:
import os , glob
import utilities.utilities as util
data_path = './matlib'
preprocess_path = './preprocess'

for filename in glob.glob(os.path.join(data_path, '*.lean')):
    util.preprocess(filename, preprocess_path, data_path)

In [1]:
import os , glob
import utilities.utilities as util

In [2]:
### Path #####
data_path = './matlib'
preprocess_path = './preprocess'
text_path = './preprocess/bilinear.txt'


#### Hyper parameter #####
prediction_type = 0 #the type of prediction
num_lines_predict = 10 #the number of lines for prediction

#### Generate txt files #######
for filename in glob.glob(os.path.join(data_path, '*.lean')):
    util.preprocess(filename, preprocess_path, data_path)

#### read txt files as list ######
text = util.list_lines(text_path)
# print('total line', len(text),'text is @@@@@@@@')
# for j in text:
#     print(j)

#### evaluate the accuracy #####
print('accuracy =', util.evaluate(text, prediction_type, num_lines_predict))
  
test01 = util.encode_text_line(text)
print(len(test01[0]))

accuracy = 0.0
102


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(102*10, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 102)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x
net = Net()
print(net)
    

Net(
  (fc1): Linear(in_features=1020, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=102, bias=True)
)


In [12]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
data_path = './matlib'
preprocess_path = './preprocess'

EPOCHS = 1
num_lines_predict = 10

for epoch in range(EPOCHS):
    l=0
    loss=0
    for filename in glob.glob(os.path.join(preprocess_path, '*.txt')): 
        text = util.list_lines(filename)
        en_text = util.encode_text_line(text)
        if l < 5:
            for i in range(len(text)-num_lines_predict-1):
                if l < 5:
                    #choose 
                    X = en_text[i:i+num_lines_predict]
                    y = en_text[i+num_lines_predict]
                    # X = []
                    # for i in chosen:
                    #     X += i
                    X = torch.FloatTensor(X)
                    y = torch.FloatTensor(y)
                    net.zero_grad()
                    output = net(X.view(-1, 102*num_lines_predict))
                    # print(y.size())
                    # print(X.view(-1,80*num_lines_predict))
                    loss = F.mse_loss(output, y)
                    loss.backward()
                    optimizer.step()
                    
            l+=1
            print(loss)

0


<ipython-input-12-d059cf8c70ee>:31: UserWarning: Using a target size (torch.Size([102])) that is different to the input size (torch.Size([1, 102])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(output, y)


tensor(1024.0367, grad_fn=<MseLossBackward0>)
tensor(4445.4863, grad_fn=<MseLossBackward0>)
tensor(3842.2334, grad_fn=<MseLossBackward0>)
tensor(123205.8438, grad_fn=<MseLossBackward0>)


In [15]:
loss

tensor(123205.8438, grad_fn=<MseLossBackward0>)

In [33]:
test = util.list_lines("./preprocess/bilinear.txt")
en_test = util.encode_text_line(test)
X = en_test[0:0+num_lines_predict]
X = torch.FloatTensor(X)
prediction = net(X.view(-1,102*num_lines_predict))
print(prediction)
prediction = [ round(i) if i>=0 else 0 for i in prediction[0].tolist()]
print(prediction)
prediction = [chr(i) for i in prediction]
print(prediction)
pred = ''
for j in prediction:
    pred+=j
pred

tensor([[  2.0231,  -6.8178,   4.6865,  22.2454,  22.9200,  -5.3504, -16.3525,
           4.1700,  -3.9402,  -3.4452,  19.4312, -19.7914, -23.6759,  11.6860,
         -11.0420,  -7.3020,  25.9670, -24.3211, -23.5097,  14.0599, -14.8022,
         -17.0320,  -1.8959,  -0.2213,  -7.6976,  26.4783,  23.7910,  31.1344,
          -1.9136,   1.1461,  11.6442,  26.0564,  -5.2490,  10.6327,  -0.0658,
         -24.1015,  -8.5875,  -5.7292,   5.0292,  19.0880,   8.4648, -15.4886,
          -4.7401,  -3.5691,  13.2633,  -9.4214,  -0.4952,  32.6685,  -2.7357,
         -14.4499, -30.2372,  11.5117, -15.9960,  16.0546,  -5.8137,  -2.8226,
          15.7915, -13.1465, -22.1337, -15.4675,  12.0241,  22.7412,   3.1383,
         -16.6246,  11.4387, -15.5147,  25.7555, -24.0560,  19.3061,  -8.5851,
          20.5422, -10.6198,  18.0356,  15.4119,   4.1431,   8.2319,   4.3209,
          -0.6170,  -4.6830,   4.1550, -10.8504,  -7.8562, -29.6704,  -5.7099,
          12.2647,   7.3766,   3.9050,   6.6082, -39

"\x02\x00\x05\x16\x17\x00\x00\x04\x00\x00\x13\x00\x00\x0c\x00\x00\x1a\x00\x00\x0e\x00\x00\x00\x00\x00\x1a\x18\x1f\x00\x01\x0c\x1a\x00\x0b\x00\x00\x00\x00\x05\x13\x08\x00\x00\x00\r\x00\x00!\x00\x00\x00\x0c\x00\x10\x00\x00\x10\x00\x00\x00\x0c\x17\x03\x00\x0b\x00\x1a\x00\x13\x00\x15\x00\x12\x0f\x04\x08\x04\x00\x00\x04\x00\x00\x00\x00\x0c\x07\x04\x07\x00\x00'\x00\x08\x00\x00#\x02\x04\x0c\x00\x00\x10"

In [134]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.tensor([1, 0, 4])
output = F.nll_loss(F.log_softmax(input), target)
output.backward()

<ipython-input-134-afb3cde887cf>:3: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.nll_loss(F.log_softmax(input), target)


In [127]:
input

tensor([[ 0.8206,  0.3732,  0.7117, -0.7908,  0.3385],
        [ 1.1699, -0.0723, -0.0484, -0.9167,  1.9843],
        [ 1.0320, -1.5377,  0.5465,  0.5768, -0.5105]], requires_grad=True)

In [24]:
import math
math.log(max(max(test01)),2)

13.237209960755022

In [26]:
pow(2,13.237209960755022)

9656.000000000005

In [27]:
chr(9656)

'▸'

In [35]:
maxi = 0
for filename in glob.glob(os.path.join(preprocess_path, '*.txt')):
    text = util.list_lines(filename)
    test01 = util.encode_text_line(text)
    newmax =  max(max(test01))
    if newmax >= maxi:
        maxi = newmax
        k = filename


In [6]:
maxi = 0
for filename in glob.glob(os.path.join(preprocess_path, '*.txt')):
    text = util.list_lines(filename)
    test01 = util.encode_text_line(text)
    j=0
    for line in test01:
        j+=1
        if len(line) >= maxi:
            maxi = len(line)
            k = filename
            l = j
            m = text[j]


In [7]:
k

'./preprocess/field_power.txt'

In [8]:
maxi

101

In [10]:
j

128

In [11]:
len('@[simp] lemma zpow_le_iff_le (hx : 1 < x) : x ^ m ≤ x ^ n ↔ m ≤ n := (zpow_strict_mono hx).le_iff_le')

100

In [ ]:
for i in range(len(text)-num_lines_predict-1):
    lines = text[i:i+num_lines_predict]
    # print(total,'th comparation between ',predict(lines, prediction_type), 'and',text[i+num_lines_predict] )
    if predict(lines, prediction_type) == text[i+num_lines_predict]:
        correct +=1

In [54]:
lists = [[1,3,5],[2,5,6],[7,8,9]]
newlist = []
for i in lists:
    newlist += i
newlist

[1, 3, 5, 2, 5, 6, 7, 8, 9]

In [53]:
newlist

[1, 3, 5, 2, 5, 6, 7, 8, 9]

In [48]:
lists[0]+lists[1]

[1, 3, 5, 2, 5, 6]

In [13]:
k = ['w','w','x']

In [14]:
k

['w', 'w', 'x']

In [15]:
k.remove('w')

In [16]:
k

['w', 'x']

In [8]:
test = util.list_lines("./preprocess/bilinear.txt")
en_test = util.encode_text_line(test)
de_test = util.decode_text_line(en_test)
test == de_test

True

In [13]:
p = os.walk('./matlib')
[x[0] for x in p]

['./matlib',
 './matlib/data',
 './matlib/data/bool',
 './matlib/data/dfinsupp',
 './matlib/data/vector',
 './matlib/data/polynomial',
 './matlib/data/polynomial/degree',
 './matlib/data/lazy_list',
 './matlib/data/string',
 './matlib/data/option',
 './matlib/data/sym',
 './matlib/data/ordmap',
 './matlib/data/zmod',
 './matlib/data/mv_polynomial',
 './matlib/data/setoid',
 './matlib/data/array',
 './matlib/data/rbmap',
 './matlib/data/multiset',
 './matlib/data/qpf',
 './matlib/data/qpf/univariate',
 './matlib/data/qpf/multivariate',
 './matlib/data/qpf/multivariate/constructions',
 './matlib/data/finset',
 './matlib/data/fun_like',
 './matlib/data/pi',
 './matlib/data/set',
 './matlib/data/set/intervals',
 './matlib/data/pnat',
 './matlib/data/set_like',
 './matlib/data/stream',
 './matlib/data/num',
 './matlib/data/fintype',
 './matlib/data/rat',
 './matlib/data/prod',
 './matlib/data/bitvec',
 './matlib/data/finsupp',
 './matlib/data/dlist',
 './matlib/data/sum',
 './matlib/data/ma

In [2]:
import glob, os
import utilities.utilities as util
data_path = './matlib'
preprocess_path = './preprocess'
p = glob.glob(data_path + '/*', recursive = True)
for path in p:
    for filename in glob.glob(os.path.join(path, '*.lean')):
        util.preprocess(filename, preprocess_path, data_path)

In [4]:
p = glob.glob(data_path + '/*', recursive = True)
p[0]

'./matlib/data'

In [3]:
from tqdm import tqdm

In [8]:
from tqdm import tqdm
lst = [1, 3, 5, 7, 11]

for element in tqdm(lst):
    a=[]

100%|██████████| 5/5 [00:00<00:00, 54471.48it/s]


In [23]:
torch.save(net,'./abe.pth')

In [24]:
net2 =  torch.load('./abe.pth')

In [25]:
net == net2

False

In [33]:
net.state_dict()['fc1.weight']

tensor([[-0.0123, -0.0312,  0.0150,  ..., -0.0179,  0.0191,  0.0024],
        [ 0.0076,  0.0264,  0.0127,  ..., -0.0313,  0.0249, -0.0239],
        [-0.0156,  0.0098, -0.0021,  ..., -0.0190, -0.0002, -0.0210],
        ...,
        [ 0.0237, -0.0251,  0.0258,  ..., -0.0249, -0.0149, -0.0072],
        [-0.0114, -0.0127,  0.0229,  ..., -0.0313,  0.0272, -0.0175],
        [ 0.0086,  0.0011,  0.0119,  ..., -0.0083,  0.0119, -0.0307]])

In [34]:
net2.state_dict()['fc1.weight']

tensor([[-0.0123, -0.0312,  0.0150,  ..., -0.0179,  0.0191,  0.0024],
        [ 0.0076,  0.0264,  0.0127,  ..., -0.0313,  0.0249, -0.0239],
        [-0.0156,  0.0098, -0.0021,  ..., -0.0190, -0.0002, -0.0210],
        ...,
        [ 0.0237, -0.0251,  0.0258,  ..., -0.0249, -0.0149, -0.0072],
        [-0.0114, -0.0127,  0.0229,  ..., -0.0313,  0.0272, -0.0175],
        [ 0.0086,  0.0011,  0.0119,  ..., -0.0083,  0.0119, -0.0307]])

In [43]:
from datetime import datetime

now = datetime.now() # current date and time
a=[now.strftime("%m-%m-%Y")]
print(a)

['07-07-2022, 00:27:58']


In [44]:
k=9
str(k)

'9'

In [45]:
score = [1,2,3,4,5]
with open('./neuralnetwork/'+now.strftime("%m-%m-%Y")+'_'+str(k)+".txt", "w") as f:
    for s in score:
        f.write(str(s) +"\n")

In [34]:
loss.item()

123205.84375

In [33]:
'loss' is str

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-33-6c3030368d97>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  'loss' is str


False

In [47]:
from datetime import datetime
now = datetime.now()
loss_list = [2.2,62,99]
k=0
while os.path.exists('./neuralnetwork/'+now.strftime("%m-%m-%Y")+'_'+str(k)+".txt"):
    k+=1
ppp = './neuralnetwork/'+now.strftime("%m-%m-%Y")+'_'+str(k)
with open(ppp+".txt", "w") as f:
    for s in loss_list:
        f.write(str(s) +"\n")

In [48]:
torch.save(net,ppp+'.pth')